# Data Preparation - IT2 - Choosing best flow 

In [ ]:
import os
import random
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# Define the path to the folder containing the images to be processed
folder_path = '../data/subset'  # Update this path to point to your specific folder containing images

## Loading Images and stats

In [ ]:
# Load image paths
def load_images_from_folder(folder_path, extensions=('.png', '.jpg', '.jpeg', '.JPG')):
    """
    Load all image file paths from a specified folder that match the given file extensions.

    Parameters:
    folder_path (str): The path to the folder containing the images.
    extensions (tuple of str): A tuple of file extensions to filter the images by. 
                               Default is ('.png', '.jpg', '.jpeg', '.JPG').

    Returns:
    list: A list of full file paths to images in the folder that match the specified extensions.
    
    Raises:
    FileNotFoundError: If the specified folder does not exist.
    """

    # Check if the folder exists
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The specified folder does not exist: {folder_path}")

    # List comprehension to gather all image paths with the specified extensions
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(extensions)]

    return image_paths


In [ ]:
# Load and preprocess images
def load_and_preprocess_images(image_paths, resize_dim=(256, 256)):
    images = []
    image_ids = []

    # Initialize tqdm progress bar
    for path in tqdm(image_paths, desc="Loading and preprocessing images", unit="image"):
        img = cv2.imread(path)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        # img_resized = cv2.resize(img_gray, resize_dim)  # Resize for consistency
        images.append(img_gray)
        image_ids.append(f'Image_{len(images)}')  # Assign image ID as Image_1, Image_2, etc.

    return images, image_ids


In [ ]:
# Load all image file paths from the specified folder
image_paths_all = load_images_from_folder(folder_path)

# Load and preprocess all images
total_images, total_image_ids = load_and_preprocess_images(image_paths_all)

# Randomly select 10 images for experimentation
experiment_indices = random.sample(range(len(total_images)), 10)
test_images = [total_images[i] for i in experiment_indices]
test_image_ids = [total_image_ids[i] for i in experiment_indices]


In [ ]:
# Load the CSV file with the image statistics
images_stats_path = "../data-understanding/images_stats.csv"
images_stats_df = pd.read_csv(images_stats_path)


## Functions of Characteristics

In [ ]:
# Image Characteristics Calculation Functions - from data understanding it2
def calculate_brightness(image):
    return np.mean(image)

def calculate_sharpness(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def calculate_contrast(image):
    return image.std()

def calculate_noise(image):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(image, (3, 3), 0)
    noise = cv2.absdiff(image, blurred)
    return np.var(noise)

# def calculate_skew(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     coords = np.column_stack(np.where(binary > 0))
#     if coords.size == 0:
#         return 0
#     angle = cv2.minAreaRect(coords)[-1]
#     if angle < -45:
#         angle = -(90 + angle)
#     else:
#         angle = -angle
#     if abs(angle) < 1e-2:
#         angle = 0
#     return round(angle, 2)
# 
# def calculate_line_spacing(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
# 
#     # Ensure the image is in the correct format for findContours
#     if image.dtype != np.uint8:
#         image = cv2.convertScaleAbs(image)
# 
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     heights = [cv2.boundingRect(contour)[3] for contour in contours]
#     if len(heights) > 1:
#         line_spacing = np.mean(np.diff(sorted(heights)))
#     else:
#         line_spacing = 0
#     return line_spacing
# 
# 
# def detect_tables(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
# 
#     # Ensure the image is in the correct format for adaptiveThreshold
#     if image.dtype != np.uint8:
#         image = cv2.convertScaleAbs(image)
# 
#     # Apply binary threshold and adaptive threshold
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     binary = cv2.adaptiveThreshold(binary, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
# 
#     # Find contours in the image
#     contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     table_contours = [contour for contour in contours if cv2.contourArea(contour) > 1000]
# 
#     return len(table_contours)
# 
# 
# def calculate_resolution(image):
#     height, width = image.shape[:2]
#     return height * width
# 
# def calculate_elements_detection(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
# 
#     # Ensure the image is in the correct format for findContours
#     if image.dtype != np.uint8:
#         image = cv2.convertScaleAbs(image)
# 
#     # Apply binary threshold
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
# 
#     # Find contours
#     contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     return len(contours)
# 
# 
# def calculate_texture(image):
#     laplacian = cv2.Laplacian(image, cv2.CV_64F)
#     return laplacian.std()
# 
# def calculate_patterns(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
# 
#     # Ensure the image is in the correct format for Canny edge detection
#     if image.dtype != np.uint8:
#         image = cv2.convertScaleAbs(image)
# 
#     # Apply Canny edge detection
#     edges = cv2.Canny(image, 100, 200)
# 
#     # Count the number of edge pixels
#     return np.sum(edges > 0)



## Evaluation Function

In [ ]:
def advanced_evaluation(image, techniques_dict, original_stats):
    evaluation_results = {}

    for technique_name, technique_func in techniques_dict.items():
        # Apply the technique
        processed_image = technique_func(image)

        # Calculate characteristics for the processed image
        stats = {
            "Brightness": calculate_brightness(processed_image),
            "Sharpness": calculate_sharpness(processed_image),
            "Contrast": calculate_contrast(processed_image),
            "Noise": calculate_noise(processed_image),
        }

        # Normalize metrics to comparable ranges (between 0 and 1, roughly)
        stats_normalized = {
            "Brightness": stats["Brightness"] / 255,
            "Sharpness": stats["Sharpness"] / 1000,
            "Contrast": stats["Contrast"] / 255,
            "Noise": stats["Noise"] / 255,
        }

        # Normalize the original stats for comparison
        original_stats_normalized = {
            "Brightness": original_stats["Brightness"] / 255,
            "Sharpness": original_stats["Sharpness"] / 1000,
            "Contrast": original_stats["Contrast"] / 255,
            "Noise": original_stats["Noise"] / 255,
        }

        # Weights for each characteristic (to determine their importance)
        weights = {
            "Brightness": 1.0,  # Higher is better (rewarded if improved)
            "Sharpness": 1.0,    # Higher is better (rewarded if improved) but images were generally sharp already 
            "Contrast": 2.0,     # Higher is better (rewarded if improved) the levels of contrast were lower and obstructed details
            "Noise": -1.5,       # Lower is better (penalized if increased)
        }

        # Calculate score using normalized metrics and weights
        score = 0
        for metric, value in stats_normalized.items():
            original_value = original_stats_normalized.get(metric, 0)
            score += weights[metric] * (value - original_value)

        evaluation_results[technique_name] = {"Score": score, "Stats": stats}

    # Determine the best technique based on the highest score
    best_technique = max(evaluation_results, key=lambda x: evaluation_results[x]["Score"])
    return {"Best Technique": best_technique, "Evaluation Results": evaluation_results}


## Setting the best techniques based on step 1.1

In [ ]:
# Define the best techniques and parameters for each step
best_techniques = {
    "Noise Reduction": (lambda img: cv2.medianBlur(img, ksize=3)),
    "Histogram Equalization": (lambda img: cv2.equalizeHist(img)),  # No parameters were found to be beneficial
    "Binarization": (lambda img: cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, blockSize=11, C=2)),
    "Morphological Operations": (lambda img: cv2.dilate(img, np.ones((9, 9), np.uint8))),
    "Edge Detection": (lambda img: cv2.Canny(img, threshold1=50, threshold2=150)),
}


## Using the previous flows found to be most suitable

In [ ]:
# Best Flows from Previous Iteration
previous_best_flows = {
    "Flow - CLAHE": [
        ("Gaussian Blur", lambda img: cv2.GaussianBlur(img, (5, 5), 0)),
        ("CLAHE", lambda img: cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(img))
    ]
}


## Setting flows based on best techniques

In [ ]:
# Create New Flows Incrementally
new_flows = {
    "Flow 1 Noise Reduction": [
        ("Noise Reduction", best_techniques["Noise Reduction"])
    ],
    "Flow 2 Noise Reduction + Histogram Equalization": [
        ("Noise Reduction", best_techniques["Noise Reduction"]),
        ("Histogram Equalization", best_techniques["Histogram Equalization"])
    ],
    "Flow 3 Noise Reduction + Histogram Equalization + Binarization": [
        ("Noise Reduction", best_techniques["Noise Reduction"]),
        ("Histogram Equalization", best_techniques["Histogram Equalization"]),
        ("Binarization", best_techniques["Binarization"])
    ],
    "Flow 4 Noise Reduction + Histogram Equalization + Binarization + Morphological Operations": [
        ("Noise Reduction", best_techniques["Noise Reduction"]),
        ("Histogram Equalization", best_techniques["Histogram Equalization"]),
        ("Binarization", best_techniques["Binarization"]),
        ("Morphological Operations", best_techniques["Morphological Operations"])
    ],
    "Flow 5 Full Flow": [
        ("Noise Reduction", best_techniques["Noise Reduction"]),
        ("Histogram Equalization", best_techniques["Histogram Equalization"]),
        ("Binarization", best_techniques["Binarization"]),
        ("Morphological Operations", best_techniques["Morphological Operations"]),
        ("Edge Detection", best_techniques["Edge Detection"])
    ]
}


## Prepearing and running the flows

In [ ]:
# Combine all flows
all_flows = {**previous_best_flows, **new_flows}

In [ ]:
# Function to apply a flow to an image
def apply_flow(flow, image):
    for step_name, technique in flow:
        image = technique(image)
    return image


In [ ]:
# Testing the Flows
flow_results = {}

# Define a specific image to save across all flows
image_to_save_id = total_image_ids[0]  # Choose the first image (or any specific image)

# Loop over each flow
for flow_name, flow_steps in all_flows.items():
    total_score = 0
    print(f"Testing Flow: {flow_name}\n{'-' * 40}")

    # Loop over each image for evaluation
    for img, img_id in zip(total_images, total_image_ids):
        original_stats = images_stats_df[images_stats_df['Image'] == img_id].iloc[0].to_dict()

        # Apply the flow on the image
        processed_image = apply_flow(flow_steps, img)

        # Save the processed image only for the selected image ID
        if img_id == image_to_save_id:
            # Specify the folder to save the processed images for the flows
            output_folder_flow = f"./Data/It2/Flows/{flow_name}"
            os.makedirs(output_folder_flow, exist_ok=True)

            # Save the processed image with the flow name and image ID
            cv2.imwrite(f"{output_folder_flow}/{flow_name}_Image_{img_id}.jpg", processed_image)

        # Using advanced evaluation function to evaluate the processed image
        evaluation_results = advanced_evaluation(processed_image, {"Flow": lambda x: x}, original_stats)

        # Access the score for the flow (since only one technique is passed, we can directly fetch it)
        flow_score = evaluation_results["Evaluation Results"]["Flow"]["Score"]

        # Add the score to the total score
        total_score += flow_score
        print(f"Image {img_id}: Score = {flow_score}")

    # Calculate the average score for the flow
    avg_score = total_score / len(total_images)
    print(f"Average Score for Flow '{flow_name}': {avg_score}\n")
    flow_results[flow_name] = avg_score


In [ ]:
# Display Final Results
sorted_results = sorted(flow_results.items(), key=lambda x: x[1], reverse=True)
print("\nFinal Flow Testing Results (Sorted by Score):")
for flow_name, score in sorted_results:
    print(f"{flow_name}: Average Score = {score}")

Final Flow Testing Results (Sorted by Score):
Flow 3: Noise Reduction + Histogram Equalization + Binarization: Average Score = 83.55
Flow 2: Noise Reduction + Histogram Equalization: Average Score = 0.51
Flow 4: Noise Reduction + Histogram Equalization + Binarization + Morphological Operations: Average Score = 0.27
Flow - CLAHE: Average Score = 0.08
Flow 1: Noise Reduction: Average Score = -0.02
Flow 5: Full Flow: Average Score = -0.69

# Flow Testing Results and Interpretation

## 1. Flow 3: Noise Reduction + Histogram Equalization + Binarization
- **Score**: 83.55
- **Steps**: Noise Reduction, Histogram Equalization, Binarization.
- **Interpretation**:
  - This flow produced the **highest average score**, indicating that combining these three techniques—**Noise Reduction**, **Histogram Equalization**, and **Binarization**—resulted in a significant improvement in image quality metrics compared to the original images.
  - **Binarization** helped make text and table lines more distinct, providing better separation of content from the background.
  - However, the absence of **Morphological Operations** and **Edge Detection** means that further refinement and edge enhancement could potentially yield even better results.

## 2. Flow 2: Noise Reduction + Histogram Equalization
- **Score**: 0.51
- **Steps**: Noise Reduction, Histogram Equalization.
- **Interpretation**:
  - Adding **Histogram Equalization** improved the results compared to just using noise reduction, suggesting that enhancing contrast played a crucial role in making text and lines stand out more.
  - The relatively low positive score indicates that although the images were improved, without **Binarization** or more advanced techniques like **Morphological Operations** or **Edge Detection**, the enhancement was limited.

## 3. Flow 4: Noise Reduction + Histogram Equalization + Binarization + Morphological Operations
- **Score**: 0.27
- **Steps**: Noise Reduction, Histogram Equalization, Binarization, Morphological Operations.
- **Interpretation**:
  - Adding **Morphological Operations** to the binarization step provided some improvement, but not as much as expected compared to Flow 3.
  - The low score indicates that while **Morphological Operations** added some structure to the images, they also introduced some level of distortion or data loss that negatively impacted the overall quality.

## 4. Flow - CLAHE
- **Score**: 0.08
- **Steps**: CLAHE.
- **Interpretation**:
  - This flow, which only used **CLAHE** (Contrast Limited Adaptive Histogram Equalization), yielded a very small positive score.
  - The limited improvement suggests that **contrast enhancement** alone, without noise reduction or edge enhancement, was not sufficient to achieve significant quality gains.

## 5. Flow 1: Noise Reduction
- **Score**: -0.02
- **Steps**: Noise Reduction.
- **Interpretation**:
  - This flow produced a **negative score**, meaning that **Noise Reduction** alone did not improve image quality and instead slightly degraded it.
  - The smoothed images lost some contrast and detail, which led to reduced image quality in terms of the evaluation metrics.

## 6. Flow 5: Full Flow
- **Score**: -0.69
- **Steps**: Noise Reduction, Histogram Equalization, Binarization, Morphological Operations, Edge Detection.
- **Interpretation**:
  - Surprisingly, the **Full Flow**—which combined all the techniques—produced the **lowest average score**.
  - This suggests that adding all five techniques together introduced too many transformations, which may have led to over-processing, resulting in the loss of important image characteristics.
  - The **Edge Detection** and **Morphological Operations** steps might have contributed to unwanted noise or artifacts, negatively affecting the overall score.

## Summary of Findings
- **Flow 3** achieved the best result, showing that a combination of **Noise Reduction**, **Histogram Equalization**, and **Binarization** is effective in enhancing the images while keeping transformations minimal.
- **Flow 2** also provided some improvement, indicating that **Histogram Equalization** is crucial for enhancing contrast and making features more prominent.
- Adding **Morphological Operations**, as in **Flow 4**, did not significantly enhance the results and might have introduced unnecessary noise or detail loss.
- The **Full Flow** (Flow 5) produced a negative score, suggesting that combining all techniques together without careful parameter tuning can lead to over-processing.
- The **best flows** balance noise reduction, contrast enhancement, and binarization to make content more distinct without over-processing the images.

Based on these results, you can conclude that a comprehensive image processing pipeline involving all the steps yields the best preparation for deep learning tasks, but simpler versions of the flow can still yield reasonable improvements depending on the requirements of the task.
